In [1]:
import numpy as np
import stock
import xueqiu
import kline
import status

# 日线能量线崛起和MACD双崛起的股票(第一买入点)

In [2]:
def isRasing(a,c,istoday):
    #istoday True可以使用xueqiu数据
    #c [0 company_id,1 code,2 name,3 category,4 ttm,5 pb]  
    #0 id ,1 close,2 volume,3 volumema20,4 macd,5 energy,6 volumeJ,7 bollup,8 bollmid,9 bolldn,10 bollw
    dMACD = a[-1,4]-a[-2,4]
    #macd在零轴附件（预计2日穿过或者已经穿过2日）,股价涨,能量线崛起
    if dMACD>0 and a[-1,1]-a[-2,1]>0 and a[-1,5]>=3 and a[-2,5]<3:
        if (a[-1,4]<0 and a[-1,4]+2*dMACD>0) or (a[-1,4]>0 and a[-1,4]-2*dMACD<0):
            return True,[{'x':[-1],'color':'magenta','linestyle':'--','linewidth':2}]
    return False,[]

def myFilter(a,c,istoday,period):
    if istoday and period=='d' and a[-1][5]<3 and status.isPopularCategory(c[3]):
        return True
    return status.isPopularCategory(c[3])

status.RasingCategoryList('d',isRasing,myFilter,name='today')

IntProgress(value=0, description='Loading:', layout=Layout(display='flex', flex_flow='wrap', width='100%'))

Output()

Box(children=(Button(button_style='danger', description='2020-02-19', style=ButtonStyle()), Button(description…

Output()

# 日线崛起后能量线在上线缩量价平二次放量上行的股票(第二买点)

In [3]:
def searchFirstBuy(a): #向前搜寻第一买入点
    c = 0
    #在本macd周期有第一买入点出现，并且当前股价没有低于第一买入收盘
    for i in range(-2,-20,-1):
        dMACD = a[i,4]-a[i-1,4]
        if dMACD>0 and a[i,1]-a[i-1,1]>0 and a[i,5]>=3 and a[i-1,5]<3:
            if (a[i,4]<0 and a[i,4]+2*dMACD>0) or (a[i,4]>0 and a[i,4]-2*dMACD<0): #第一买入点找到
                return i
        if a[i,4]<0:
            c += 1
            if c>=2:
                break
    return 0
def isRasing2(a,c,istoday):
    #istoday True可以使用xueqiu数据
    #c [0 company_id,1 code,2 name,3 category,4 ttm,5 pb]
    #0 id ,1 close,2 volume,3 volumema20,4 macd,5 energy,6 volumeJ,7 bollup,8 bollmid,9 bolldn,10 bollw
    first = searchFirstBuy(a)
    level = 1 #阶段
    #从第一买点，可以经历量增（阶段1），量减（阶段2），随后的第一个量增（阶段3）。
    #阶段1忽略，阶段2如果发生量缩价增提示，阶段3提示
    if first!=0:# and a[-1,2]/a[i,2] > a[-1,1]/a[i,1]:
        volumeJ = a[-1,6]
        if volumeJ<70 and a[-1,2]<a[-2,2] and a[-1,1]>a[-2,1]:
            return True,[{'x':[-1],'color':'blue','linestyle':'--','linewidth':2},{'x':[first],'color':'magenta','linestyle':'--','linewidth':2}]
    return False,[]

def myFilter2(a,c,istoday,period):
    #macd>0 能量线大于3没有完全下线
    if period=='d' and a[-1,4]>0 and a[-1,5]>3 and status.isPopularCategory(c[3]):
        i = searchFirstBuy(a)
        if i!=0 and a[i,1]<a[-1,1]: #找到第一买入点，并且当前价格没有低于第一买入点
            return True
    return False
status.RasingCategoryList('d',isRasing2,myFilter2,name='today2')

IntProgress(value=0, description='Loading:', layout=Layout(display='flex', flex_flow='wrap', width='100%'))

Output()

Box(children=(Button(button_style='danger', description='2020-02-19', style=ButtonStyle()), Button(description…

Output()